My subsort checker database is basically a congryuence database?

cong = {
    set.Union: kd.ForAll([A,B,C,D], A <= B, C <= D, Union(A,C) <= Union(B, D))
    set.Inter:
}



Var trick for opening binders? Is it sound?
Use original lambda as "name" of variable.
lam(var(i) + 1) -->  fvar(lam(var(i) + 1)) + 1
Don't use numbers when you don't have to
"Skolemization"
"Use the reason it exists"
"provenance"

Analog of the `let` trick.
The free varaible was certainly "caused" by opening the binder.

Is 0*bvar(0) = 0 a refinement? Kind of feels that way (if looking up a variable is )
0*bvar(0) --> 0
It errors out in strictly less contexts.


The point of refinment/undef behavior is to enable more rewriting.

https://archive.computerhistory.org/resources/access/text/2024/01/102805327-05-01-acc.pdf Henglein "semi-unification"

https://pure.mpg.de/rest/items/item_1834874_3/component/file_1857517/content Is this the same or a different semi unification?
https://www.sciencedirect.com/science/article/pii/0304397591901899 Kapur semi unification
sig phi t = phi s
a unification and a matching
https://types.pl/@flippac/115102336973734248

or sig t <= sig s  equivalently. 

https://dl.acm.org/doi/10.1145/169701.169687 type reconstruction in the presence of polymorphic recursion
https://cormack.uwaterloo.ca/rasup.pdf  A Larger Decidable Semiunification Problem

 Amortized efficiency of a path retrieval data structure
https://www.sciencedirect.com/science/article/pii/0304397586900988 

https://chaozhang-cs.github.io/files/sigmod23-tutorial-short.pdf An Overview of Reachability
Indexes on Graphs
- tree cover indices
- 2-hop indices - hmm. Kind of like a union find to root, away from root. Finding good quasi roots
- approximate reachability

This does match my paradigm of rebuild or approximate.

https://drops.dagstuhl.de/storage/00lipics/lipics-vol221-sand2022/LIPIcs.SAND.2022.1/LIPIcs.SAND.2022.1.pdf Recent Advances in Fully Dynamic Graph
Algorithms


https://github.com/facebook/flow/blob/main/src/typing/tvar_resolver.ml 
https://github.com/facebook/flow/blob/d076b30cfc3635ff8c38d43bee0b08c66cb9ec4f/src/typing/context.ml#L1209 find_root? I dunno. This seems problematic.


That lattice textbook


Where in compilers is there an example of a contravaraint symbol?
Inputs to function calls?
Call site f(x,y,z)
bind site
def f(x,y,z):  

t <= pat <= rhs

`t <bisubst= pat <= rhs




E-graph trinitarianism - hash cons <-> union find <-> egraph

Ordered chaining
https://www.philipzucker.com/le_find/

```
tSu          vSw
-----------------
  sig[t] S sig[w]
```

But with the usual restrction that we only need to do inferences that increase something?


ground ordered chaining. maybe there is some path compression that can work here?


https://inria.hal.science/inria-00073205/document Pottier thesis


Higher order Bi-Unfication
F(x) <= x

https://docs.google.com/document/d/15amCalh9CSOWbbZ3d_haNad0Pw_jJ7vYcjMLkp7r-AA/edit?tab=t.0


icfp gilbert something knitting groupoid. Hmm.

2 more
- refinement reasoning in algerab of programming. Relational spec. functional implementation. The calculated typer https://people.cs.nott.ac.uk/pszgmh/typer.pdf . Program Construction: Calculating Implementation  from Specifications . Mathmeth http://www.mathmeth.com/ https://www.cs.ox.ac.uk/publications/books/PfS/ . Gries?
- Cycluic proofs.  We get the theoremwe're trying to rpover as a inequality guarded rule. Would not numerical ineq be good enough?1

https://en.wikipedia.org/wiki/Relation_algebra
https://www.philipzucker.com/a-short-skinny-on-relations-towards-the-algebra-of-programming/
https://www.di.uminho.pt/~jno/ps/pdbc.pdf
Gibbons book?
agda aop
Backhouse calculation book

https://staff.math.su.se/anders.mortberg/papers/refinements.pdf refinement for free
dense poly to sparse poly
https://link.springer.com/content/pdf/10.1007/978-3-031-57262-3_10.pdf  Trocq: Proof Transfer for Free, With or Without Univalence 2024. Hmm. 2 versions of bitvec
CoqEAL library https://github.com/rocq-community/coqeal

Automatic and Transparent Transfer of
Theorems along Isomorphisms
in the Coq Proof Assistant  Theo Zimmermann1 and Hugo Herbelin 2015

Data vs Program rtefinement. Cahnge the program or change the datatype.
quot(X) = Y  fine. mod out permutations.
canon(Y) = X , pick one somewhat arbitrarily vs
Y -> X    . Kind of expressing containment in quoitent set. It is possible to refine Y to many possible X.




Dénès, M., Mörtberg, A., Siles, V.: A refnement-based approach to computational
algebra in coq

Generaliuzed rewriting https://rocq-prover.org/doc/v8.18/refman/addendum/generalized-rewriting.html 
https://jfr.unibo.it/article/download/1574/1077/3383 A New Look at Generalized Rewriting in Type Theory Sozeau
Subst (being <=) and diff being a fun symbol. That's a good one. since diff is `->` in some heyting sense, also makes sense.

Indeed A cup B <= C  is the same as A <= C  and B <= C . Is that useful? Does that fight an AC problem?
So set algebra. A = B, C = D,  diff, union, complement, intersection . Some of these do have anti modality. 
set constraints
clp set
{a} <= A  is open set with a in it.  elem(a,A) == sing(a) <= A

"mediated" equality is very much exactly gewneralized rewriting in sozeau sense
The variance / compatibility rules are the custom congruence rules.

[Bas94] David A. Basin. Generalized Rewriting in Type Theory
A Semi-reflexive Tactic for (Sub-)Equational Reasoning  Claudio Sacerdoti Coen

Andrew McCreight. Practical tactics for separation logic
Nick Benton and Nicolas Tabareau. Compiling Functional Types to Relational Specifications for Low Level Imperative Code. used generalized rewriting for their tactics?


https://maude.cs.illinois.edu/w/images/0/0f/BMgrt_2003.pdf  Generalized Rewrite Theories -Roberto Bruni12 and Jos´e Mesegue . Maude. Kind of hartd to read. This might be getting at some of the same stuff. Yes.  E and R.


https://en.wikipedia.org/wiki/Refinement_calculus




In [ ]:
from kdrag.all import *
State = smt.DeclareSort("State")
Rel = smt.ArraySort(State, State, smt.BoolSort()) # Pair -> Bool?

comp = smt.Function(".", Rel, Rel, Rel)
kd.notation.matmul.register(Rel, comp)
id = smt.Const("id", Rel)
R,S,T = smt.Consts("R S T", Rel)
id.left = kd.axiom(smt.ForAll([R], id @ R == R))
id.right = kd.axiom(smt.ForAll([R], R @ id == R))
comp.assoc = kd.axiom(smt.ForAll([R, S, T], (R @ S) @ T == R @ (S @ T)))

le = smt.Function("<=", Rel, smt.BoolSort())
kd.notation.le.register(Rel, le)

le.trans = kd.axiom(kd.QForAll([R, S, T], le(R, S) & le(S, T), le(R, T)))
le.refl = kd.axiom(kd.QForAll([R], le(R, R)))
le.antisym = kd.axiom(kd.QForAll([R, S],
    le(R, S) & le(S, R) >> (R == S)))

# I could define them, but yolo

Set = smt.ArraySort(State, smt.BoolSort())
dom = smt.Function("dom", Rel, Set)
cod = smt.Function("cod", Rel, Set)

conv = smt.Function("conv", Rel, Rel)
conv.id = kd.axiom(smt.ForAll([R], conv(id) == id))
conv.comp = kd.axiom(smt.ForAll([R, S], conv(R @ S) == conv(S) @ conv(R)))
conv.inv = kd.axiom(smt.ForAll([R], conv(conv(R)) == R))

join = smt.Function("join", Rel, Rel, Rel)
kd.notation._or.register(Rel, join)
join.comm = kd.axiom(smt.ForAll([R, S], R | S == S | R))
join.assoc = kd.axiom(smt.ForAll([R, S, T], (R | S) | T == R | (S | T)))
join.le = kd.axiom(smt.ForAll([R, S], R <= R | S))
join.univ = kd.axiom(smt.ForAll([R,S,T], ((R | S) <= T) == (R <= T) & (S <= T)))

top = smt.Const("top", Rel)
bot = smt.Const("bot", Rel)

const = smt.Function("const", State, Rel)




# Setlog and set constraints
See clp_set notes



In [1]:
%%file /tmp/stuff.pl

hello("world").


Writing /tmp/stuff.pl


In [10]:
! scryer-prolog /tmp/stuff.pl -g 'hello(X), write(X), halt.'

[w,o,r,l,d]

In [24]:
%%file /tmp/minikan.rkt
#lang racket
(require minikanren)

(run* (q) (== q 'hello-world))
(run* (q) (fresh (x) (== x 'hello) (== q x)))

Overwriting /tmp/minikan.rkt


In [25]:
! racket /tmp/minikan.rkt

'(hello-world)
'(hello)


# Refinement Closure
---
title: Congruence Closure becomes Refinement Closure for Refinement E-graphs
---
Using the term E-graph maybe is a bit of a misnomer.

Even in refinement reasoning, equalities are pretty prevalent.

Congruence closure generalizes to refinement closure

cong close saturates
x = y -> f(x) = f(y)


ref closure saturates
x <= y -> f(x) <= f(y) for monotonic
x <= y -> f(y) <= f(x) for anti-monotonic


x0 <=^p0 y0, x1 <=^p1 y1, ... -> f(x0,x1,x2,...) <= f(y0, y1, y2,...)


Where pn are the polarity of position n in the function symbol f.

Arrow in subtyping has polarity signature
arr(-,+)




parents and upper are similar beasts kind of.

Is there a relationship between the dpeendent egraph and refinement egraph. Types kind of givey ou a handle on partiality.


Yea, maybe this is a point towards egglog.


In [ ]:
class REGraph():
    def __init__(self):
        self.enodes = {}

    def rebuild(self):
        # The quadatic loop version
        for enode1 in enodes:
            for node2 in enodes:
                if all(self.uf.is_le(x,y) in zip(enode.args, enodes1.args)):
                    self.uf.assert_le(enode1, enode2)
        
        for enode1 in enodes:
            # search through all upper sets of args
            # we can prune though things with smallest args set
            # indeed, good joins cure a lot of wounds.
        
        # alternative: Fill out the upper set of args
        for enode in enodes:
            for a in self.uf.le_set(enode.args[0]):
                e1 = self.enodes.get_make(enode with a)
                self.uf.assert_le(a, e1)
            






# Datalog Model
Datalog / Relational / Flattened model

Egglog also has datalog model. brute force eq(x,y) relation with symmettry and transitivity moves.

Refinement closure can either brute fore

```
f(x,y) ~  f(x,y,res) relation

% refinement closure

% don't generate new symbols
 le(z,z1) :- f(x, z), le(x, y), f(y, z1)

% Do generate new f ids
set f(y) <= z :- f(x) = z, x <= y

```

Generalized rewriting coq
FOLDS.
Lessons that baking in boring congruence may not be what you want. 





# Ground Ordered Chaining



In [ ]:
type le = list[tuple[object,object]]



## Chain UF

The idea of an approximate LE via carefully picking how parents work is interesting.
Is there a domain where this can be thought of as precise? Not general partial order.
Maybe it can be precise if we guarantee a maximum width?



In [ ]:
# do you best to try and store good chains. https://en.wikipedia.org/wiki/Partially_ordered_set#Derived_notions
# chains are kind of like parent
class LEFind():
  chain0 : list[int]
  chain1 : list[int]

  def union(self, a, b): ...
    # seek lowest common ancestor?

  # .. chain

# biunify kata

Everything I can do with terms
- matching
- unification
- egraphs
- knuth bendix
- subterm


Can i extend to "bi" versions?

- BiProlog . BiKanren

Based on the LEfind, should bi substutuitions be done as
{eq : , upper : [] , lower : []}
Presumably the equality case is more common (?) so should be pulled out. 
`(<= a b)` in minikanren would branch or assert?
bi is the analog of e-unification in many respects.
So look more towards FLP for inspiration?

Would LEFind be useful for a CLP(Set)?




In [ ]:
type bisubst = tuple[dict,dict]
def apply(pat, bisubst): ...
def bimatch(t, pat): ...
def biunify(): ...




In [ ]:
from kdrag.all import *
def le_unify(t1, t2): 
    upsubst : dict[object, list[object]]= {} # nondet substitution
    downsubst = {}
    eqsubst : dict[object, object] = {} # equality substitution

    todo = [[(t1,t2)]]
    while todo:



# smt egraph

In [1]:
from dataclasses import dataclass
from collections import defaultdict
@dataclass
class LEFind():
    parents : dict
    upper : defaultdict(set)
    lower : defaultdict(set)
    def __init__(self):
        self.parents = {}
        self.upper = defaultdict(set)
        self.lower = defaultdict(set)
    def assert_le(self, x, y): # assert to LEFind that x <= y
        x,y = self.find(x), self.find(y)
        if x == y:
            return
        self.upper[x].add(y)
        self.lower[y].add(x)
        if self.is_le(y, x): # propagate antisymmettry x <= y and y <= x implies x == y
            self.union(x, y)
            for z in self.le_set(x) & self.ge_set(y): # anything between the two is squeezed
                self.union(z, y)
            for z in self.le_set(y) & self.ge_set(x): # anything between the two is squeezed. Is this redundant?
                self.union(z, x)
    def assert_ge(self, x, y): # assert to LEFind that x >= y
        self.assert_le(y, x)
    def union(self, x, y): # assert that x == y
        x, y = self.find(x), self.find(y)
        if x != y:
            self.parents[x] = y # refular union find
            self.upper[y].update(self.upper[x]) # merge upper sets
            self.lower[y].update(self.lower[x]) # merge lower sets
    def find(self, x : int) -> int:
        while x in self.parents:
            x = self.parents[x]
        return x
    # The next 3 functions are very similar. is_le can early stop when it hits y.
    def is_le(self, x, y) -> bool:
        # DFS search for y in upper set of x
        x,y = self.find(x), self.find(y)
        if x == y:
            return True
        todo = [x]
        seen = set(todo)
        while todo:
            x = todo.pop() # invariant is that x is already representative
            for z in self.upper[x]:
                # Is there a way to use lower set for pruning?
                z = self.find(z)   # compression could be updating z in place in upper[x]
                if z == y:
                    return True
                elif z not in seen:
                    seen.add(z)
                    todo.append(z)
        return False
    def le_set(self, x) -> set[int]: # all solutions to x <= ?
        x = self.find(x)
        todo = [x]
        seen = set(todo)
        while todo:
            x = todo.pop()
            for z in self.upper[x]:
                z = self.find(z)
                if z not in seen:
                    seen.add(z)
                    todo.append(z)
        return seen
    def ge_set(self, x) -> set[int]: # all solutions to x >= ?
        x = self.find(x)
        todo = [x]
        seen = set(todo)
        while todo:
            x = todo.pop()
            for z in self.lower[x]:
                z = self.find(z)
                if z not in seen:
                    seen.add(z)
                    todo.append(z)
        return seen

Hmm. You know, if there is no further interpretation of le, Having the solver backed version does not help anything.

FALSE. Kind of. Refinement closure. But we can't express monotonicty to smt without using quantiufiers.

forall x <= y -> f(x) <= f(y)

There is also an issue that I don't really know how to encode a model of undefinedness into SMT.
The relational model? That's more like the contextual egraph right?
The option model?



In [ ]:
from kdrag.solvers.egraph import EGraph

class LEGraph(EGraph):
    def __init__(self):
        super().__init__()
        self.lefind = LEFind()
        self.le_prim = smt.Function("le", smt.IntSort(), smt.IntSort(), smt.BoolSort())
        self.le = smt.TransitiveClosure(self.le_prim)
    
    def assert_le(self, x, y):
        self.lefind.assert_le(x, y)
        self.solver.add(self.le_prim(x, y))

    def is_le(self, x, y):
        return self.lefind.is_le(x, y)
        with self.solver:
            self.solver.add(smt.Not(self.le(x, y)))
            return self.solver.check() == smt.unsat
    def le_set(self, x):
        return self.lefind.le_set(x)





In [ ]:
@dataclass
class EGraph():
    lefind : LEFind
    enodes : dict

